In [1]:
import logging
import numpy as np

from wave_monitor import WaveMonitor
h = logging.StreamHandler()
h.setLevel(logging.DEBUG)
h.setFormatter(logging.Formatter("%(levelname)s %(message)s"))
WaveMonitor.logger.addHandler(h)

monitor = WaveMonitor()

t = np.linspace(0, 1, 1_000_001)  # 1m pts ~= 1ms for 1GSa/s.
f = 5
i_wave = np.cos(2 * np.pi * f * t)
q_wave = np.sin(2 * np.pi * f * t)

WaveMonitor.logger.setLevel(logging.DEBUG)
monitor.add_wfm("wave_1", t, [i_wave, q_wave])
WaveMonitor.logger.setLevel(logging.INFO)

DEBUG msg to send: {'_type': 'get_wfm_interval'}
DEBUG Wrote 24 bytes, 0 bytes remaining.
DEBUG query received: {'_type': 'wfm_interval', 'interval': 0}
DEBUG Adding waveform 'wave_1'
DEBUG msg to send: {'_type': 'add_wfm', 'name': 'wave_1', 't': array([0.00000e+00, 1.00000e-06, 2.00000e-06, ..., 9.99998e-01,
       9.99999e-01, 1.00000e+00], shape=(1000001,)), 'ys': [array([1., 1., 1., ..., 1., 1., 1.], shape=(1000001,)), array([ 0.00000000e+00,  3.14159265e-05,  6.28318530e-05, ...,
       -6.28318530e-05, -3.14159265e-05, -1.22464680e-15],
      shape=(1000001,))]}
DEBUG Wrote 24000201 bytes, 0 bytes remaining.


In [2]:
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave], dtype=None)

33.6 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave], dtype=np.float32)

The slowest run took 4.21 times longer than the fastest. This could mean that an intermediate result is being cached.
31.6 ms ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%timeit monitor.get_wfm_interval()

65.2 ns ± 0.725 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
mon2 = WaveMonitor()
mon2.get_wfm_interval = lambda: 0  # patch to avoid the delay.
%timeit mon2.add_wfm("wave_2", t, [i_wave, q_wave])

C:\Users\qiujv\dev\WaveMonitor\src\wave_monitor\client.py:212: UserWarning: Monitor is already running, not starting a new one.
  warnings.warn("Monitor is already running, not starting a new one.")


37.8 ms ± 4.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
from PySide6.QtNetwork import QLocalSocket
class FakeSocket:
    def write(self, data: bytes) -> int:
        return len(data)
    
    def waitForBytesWritten(self) -> bool:
        return True
    
    def state(self):
        return QLocalSocket.ConnectedState
    
mon2.sock = FakeSocket()  # patch to avoid the delay.
%timeit mon2.add_wfm("wave_2", t, [i_wave, q_wave])

21.6 ms ± 843 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave])

The slowest run took 4.12 times longer than the fastest. This could mean that an intermediate result is being cached.
52.8 ms ± 32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
from viztracer import VizTracer
with VizTracer(output_file="result.json") as tracer:
    monitor.add_wfm("wave_1", t, [i_wave, q_wave], dtype=np.float16)

Total Entries: 123                                                              
Use the following command to open the report:
vizviewer c:\Users\qiujv\dev\WaveMonitor\notebooks\result.json


In [10]:
# close the window now.
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave])

35.1 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
monitor.refresh_connect()

True

In [12]:
monitor.sock.state()

<LocalSocketState.ConnectedState: 3>